## Preparatory work

We first load the necessary backend and the necessary library for plot to work correctly and to to our computations.

In [1]:
import numpy as np
import re
import os
from sklearn import mixture
from scipy import stats
from os import listdir
from os.path import isfile, join
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt


We wrote here a function to read the Amerge.txt file.

In [2]:
def read_data(letter : str):

    path = './Unistroke/'+letter+'merge.txt'
    with open(path, 'r') as f:
        nb_line = sum(1 for _ in f)
    with open(path, 'r') as f:
        data = np.zeros((nb_line,2))
        i = 0
        for line in f:
            data[i][0] = float(line.strip().split(" ")[0])
            data[i][1] = float(line.strip().split(" ")[1])
            i+=1
        return data

def read_and_merge(letter : str):
    files = os.listdir('./Unistroke/')
    regex = re.compile(r'^'+letter+'[0-9]')
    files = list(filter(regex.search, files))
    data = list()
    for file in files:
        with open("./Unistroke/"+file) as content: 
            for line in content:
                data.append((float(line.strip().split("\t")[0]),float(line.strip().split("\t")[1])))

    return np.array(data)

$\textbf{1. Simulate a sample of size 500 of the following bivariate GMM :}$

$$ 0.3 N( \mu_1,\Sigma_1)+ 0.7 N(\mu_2,\Sigma_2) $$

In [3]:
mean1 = [-3,0]
cov1 = [[5,-2],[-2,1]]

mean2 = [3,0]
cov2 = [[5,2],[2,2]]

d1 = np.random.multivariate_normal(mean1, cov1, 500)
d2 = np.random.multivariate_normal(mean2, cov2, 500)

mixt = np.concatenate((d1,d2))


plt.plot(mixt[:,0], mixt[:,1],".")
plt.title('Simulated gaussian mixture')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

We note that the simulated gaussian mixture looks like the figures in the slides of the class.

## Data analysis: Gaussian model

In this section, we analyse data coming from the Unistroke data set. 
We would like to know if a bivariate Gaussian mixture can make for a good representation of the letter A.

We use the mixture package from the library sklearn to avoid implementing ourselves the EM algorithm for estimating parameters of a Gaussian mixture.

$\textbf{1. Estimate a bivariate GMM on the letter A data set and provide the estimated parameters.}$



  We first read and plot the letter A data set after angular transformation. The transformation is the following :
  
  $$ x'_n = \frac{x_{n+1}-x_n}{|x_{n+1}-x_n|}$$
  
  We understand why the data points appear on a circle using this transformation as $|x'_n|=1$

In [4]:
data = read_data("A")
plt.figure()
plt.plot(data[:,0], data[:,1],'.')
plt.title('Merging of all A files after angular transformation')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


To make the estimation we use the function fit() from the gaussian\_mixture
package. The only parameter we define is the number of components of the Gaussian mixture, here n\_components=2. The algorithm fits the model using an EM algorithm for Gaussian mixtures.
The stopping criterion is reached when the change of likelihood between two iteration is less than $\textbf{tol}$. The maximum number of iteration is $\textbf{max_iter}$. The initialisation process is assured internally by the method \_initialize\_parameters(self , X, random\_state). We can see in this method that the initialisation process can be done using the k-mean algorithm or using random initial parameters. by default it will use a k-mean algorithm.
We chose to let every parameters set as the default ones.
We chose here to let the full freedom to the covariance matrices (option full).
The algorithm return the following estimated parameters for our data :

In [5]:
#EM algorithm for gaussian mixtures
gmm = mixture.GaussianMixture(2,covariance_type='full')
gmm.fit(data)
print("means : ",gmm.means_)
print("\n")
print("covs : ",gmm.covariances_)

means :  [[-0.36571287  0.9060224 ]
 [-0.29510373 -0.72483125]]


covs :  [[[0.03903012 0.01426564]
  [0.01426564 0.00634938]]

 [[0.27188755 0.0823439 ]
  [0.0823439  0.1156479 ]]]


The results obtained are the means and the covariance matrices of each of our 2 estimated Gaussian after the EM algorithm.

$\textbf{2. Label the data using the estimated model and show the pdf of the estimated GMM}$

To label the data we use the method predict from the Gaussian mixture.
The predict method use the fitted parameters from the function fit to estimate the probability of each data point to belong to one cluster or the other. 

In [6]:
# 2
# meshgrid
labels = gmm.predict(data)
x = np.linspace(-2,2,500)
X,Y = np.meshgrid(x,x)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y
# pdf
rv = stats.multivariate_normal(gmm.means_[0],gmm.covariances_[0])
rv2 = stats.multivariate_normal(gmm.means_[1],gmm.covariances_[1])
# color with classification
bool_label= list(map(bool,labels))
inv_bool_label = [not i for i in bool_label]  
data1 = data[bool_label]
data2 = data[inv_bool_label]
# plot
fig, axs = plt.subplots(1, 2, figsize=(8, 4),constrained_layout=True)

axs[0].contourf(X,Y,rv.pdf(pos))
axs[0].plot(data1[:,0], data1[:,1],'.', color='g')
axs[0].plot(data2[:,0], data2[:,1],'.', color='b')
axs[0].set_title('fitted distribution of blue data points')
axs[0].set_xlabel('x')
axs[0].set_ylabel('y')

axs[1].contourf(X,Y,rv2.pdf(pos))
axs[1].plot(data1[:,0], data1[:,1],'.', color='g')
axs[1].plot(data2[:,0], data2[:,1],'.', color='b')
axs[1].set_title('fitted distribution of green data points')
axs[1].set_xlabel('x')
axs[1].set_ylabel('y')

fig.suptitle('classification of data points and \n gaussian pdf estimated by the EM algorithm',fontsize=15)
fig.show()

$\textbf{3 (a) : Plot each marginal histogram and add the estimated mixture of univariate
Gaussian pdfs to the figure.}$



In [7]:
fig2, axs2 = plt.subplots(1, 2, figsize=(8, 4),constrained_layout=True)

axs2[0].hist(data[:,0],150,  density = True)
axs2[1].hist(data[:,1],150,  density = True)

mg = stats.norm(gmm.means_[0,0], gmm.covariances_[0,0,0])
mg2 = stats.norm(gmm.means_[0,1], gmm.covariances_[0,1,1])
mg3 = stats.norm(gmm.means_[1,0], gmm.covariances_[1,0,0])
mg4 = stats.norm(gmm.means_[1,1], gmm.covariances_[1,1,1])

axs2[0].plot(x,(mg.pdf(x)+mg3.pdf(x))*0.5)
axs2[1].plot(x,(mg2.pdf(x)+mg4.pdf(x))*0.5)
axs2[0].set_title('marginal along x')
axs2[1].set_title('marginal along y')

axs2[0].set_xlabel('x')
axs2[0].set_ylabel('density')
axs2[1].set_xlabel('y')
axs2[1].set_ylabel('density')

fig2.suptitle('marginal histograms and marginal \n estimated gaussian mixtures',fontsize=15)
fig2.show()

$ \textbf{3. (b) For each marginal, provide separate histograms of each cluster and add the estimated
 Gaussian pdf to the figure.}$

In [8]:
fig3, axs3 = plt.subplots(2, 2, figsize=(8, 4),constrained_layout=True)
bar_num = 40
axs3[0][0].hist(data1[:,0],bar_num,  density = True,color='g')
axs3[0][0].plot(x,(mg3.pdf(x)),color='r')
axs3[0][0].set_title('marginal along x, green cluster')

axs3[0][1].hist(data1[:,1], bar_num,  density = True,color='g')
axs3[0][1].plot(x,(mg4.pdf(x)),color='r')
axs3[0][1].set_title('marginal along y, green cluster')

axs3[1][0].hist(data2[:,0],bar_num,  density = True,color='b')
axs3[1][0].plot(x,(mg.pdf(x)),color='r')
axs3[1][0].set_title('marginal along x, blue cluster')

axs3[1][1].hist(data2[:,1],bar_num,  density = True,color='b')
axs3[1][1].plot(x,(mg2.pdf(x)),color='r')
axs3[1][1].set_title('marginal along y, blue cluster')

axs3[0][0].set_xlabel('x')
axs3[0][0].set_ylabel('density')
axs3[1][0].set_xlabel('x')
axs3[1][0].set_ylabel('density')
axs3[0][1].set_xlabel('y')
axs3[0][1].set_ylabel('density')
axs3[1][1].set_xlabel('y')
axs3[1][1].set_ylabel('density')


fig3.suptitle('marginal estimated pdf and histogram for each cluster ',fontsize=15)
fig3.show()

AttributeError: module 'matplotlib.pyplot' has no attribute 'cls'

$\textbf{Plot each data point x i with some colourmap corresponding to}$ $P(Z_i = 1|X_i )$

In [ ]:
posterior_proba=gmm.predict_proba(data)
plt.scatter(data[:,0], data[:,1], c=np.log(posterior_proba[:,0]))
plt.title('data points given posterior probability')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

To compute $P(Z_i = 1|X_i )$, we use the predict_proba() function. It returns for each fitted gaussian the probability of each data points to be generated by the said Gaussian. This can be interpreted as a "soft" or continuous labelling of the data points. We see on the plot above a clear dichotomy between two sets of data points corresponding to the two different labelled clusters. This was the expected results. Using the log of the posterior probability helps for visualisation as the transition between the two cluster is too "sharp" without it.

In [ ]:
posterior_proba=gmm.predict_proba(data)
plt.scatter(data[:,0], data[:,1], c=posterior_proba[:,0])
plt.title('data points given posterior probability')
plt.xlabel('x')
plt.ylabel('y')
plt.show()